In [1]:
import sys
sys.path.insert(0, "..")

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

from utils import configure_pandas
from preprocess import spacy_tokenizer
from preprocess import stopwords

configure_pandas()

In [3]:
DATAPATH = "../data/raw/data/"
df = pd.read_csv(f"{DATAPATH}ton.csv", low_memory=False)

In [4]:
# Test custom tokenizer
example = df.loc[0, "text"][:1000]
print(example)
print("\n=== Tokenized ===")
tokens = spacy_tokenizer(example)
print(" ".join(tokens))

Ærede medrepresentanter! Tidligere stortingsrepresentant Sjur Lindebrække er død – 89 år gammel. Etter juridisk embetseksamen i 1931 var Sjur Lindebrække først dommerfullmektig i Nordfjord og deretter universitetsstipendiat i rettsvitenskap. Selv om jusen som vitenskap ikke ble hans karriere, var Sjur Lindebrække en fremragende jurist. I 1940 fikk han Kongens gullmedalje for skriftet «Eiendomsrettens overgang ved frivillig overdragelse av løsøre», og i 1948 tok han doktorgraden på avhandlingen «Eiendomsrett og konkursbeslag». I 1936 ble Lindebrække ansatt i Bergens Privatbank. Bortsett fra årene i Stortinget, kom banken til å bli hans yrkesmessige hjem. I ulike perioder av sitt liv besatt Lindebrække omtrent alle posisjonene i banken. Han var banksjef, formann i bankens styre, arbeidende styreformann og administrerende direktør. I tillegg var han i en periode formann i Den norske bankforening. Sjur Lindebrække ble valgt til Stortinget høsten 1945, som representant for Høyre i Bergen. P

In [5]:
# Remove outlier text lengths
df = df[df["text"].str.len() < 3000]
print(len(df))

# Limit for faster runs
# N = 1000
# df = df.loc[:N, :]

216922


## Bag of Words

In [6]:
vectorizer = CountVectorizer(
    tokenizer=spacy_tokenizer,
    analyzer="word",
    max_df=0.9,
    min_df=5,
    ngram_range=(1, 2),
)
data_vectorized = vectorizer.fit_transform(df["text"])
print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names()[:20])

403672
['', ' ', ' a', ' abid', ' absolutt', ' adgang', ' advare', ' aetat', ' afghanistan', ' aftenposte', ' agde', ' akershus', ' akkurat', ' akseptere', ' aksje', ' aksjeselskap', ' aktiv', ' aktivitet', ' aktuell', ' aktør']


## Topic Model: Latent Dirichlet Allocation

In [7]:
num_topics = 10
lda_model = LatentDirichletAllocation(
    n_components=num_topics,
    max_iter=10,
    learning_method="online",
    verbose=True,
)

data_lda = lda_model.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [8]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/home/eivind/.local/share/virtualenvs/nlp_test-GlIBI5hj/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=            x       y  topics  cluster   Freq
topic                                        
6      32.421   8.115       1        1 19.990
7       0.649  71.707       2        1 17.339
5      71.020   9.971       3        1 14.417
3       3.786  33.185       4        1 10.121
4     -42.667  -9.222       5        1  9.934
1      -3.622 -43.527       6        1  8.259
2      -3.632  -4.172       7        1  6.664
9     -36.584  35.804       8        1  5.470
8      44.391  50.615       9        1  3.961
0      40.236 -31.592      10        1  3.843

[10 rows x 5 columns], topic_info=                Term       Freq      Total Category  logprob  loglift
0                    250154.000 250154.000  Default   30.000   30.000
99548        forslag  87205.000  87205.000  Default   29.000   29.000
272592  representant  87524.000  87524.000  Default   28.000   28.000
63329            ein  55588.000  55588.000  Default   27.000   27.000
60008             eg  49618.000  49618.000  Default   26.000   26.000
...              ...        ...        ...      ...      ...      ...
248785           per   2293.564   4889.599  Topic10   -5.824    2.502
219610          møte   3776.463  20010.191  Topic10   -5.325    1.591
0                      7475.394 250154.790  Topic10   -4.642   -0.252
92805      foreligge   2017.131   6574.328  Topic10   -5.952    2.077
112895         følge   1744.947  30392.819  Topic10   -6.097    0.401

[645 rows x 6 columns], token_table=        Topic  Freq                 Term
term                                    
0           3 0.161                     
0           4 0.000                     
0           5 0.021                     
0           6 0.426                     
0           7 0.000                     
...       ...   ...                  ...
402364      1 1.000               – møte
402512     10 1.000  – oppfølgingsspø...
402920      2 0.944                 – si
402920      3 0.008                 – si
402920      4 0.048                 – si

[1427 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 6, 4, 5, 2, 3, 10, 9, 1])